In [ ]:
from dotenv import load_dotenv
from tqdm import tqdm
from openai import OpenAI
import anthropic
import google.generativeai as genai
import PIL.Image
import base64
import os
import time
import pandas as pd
import numpy as np

In [ ]:
load_dotenv()

In [3]:
chat_gpt = OpenAI()
claude = anthropic.Anthropic()
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
gemini = genai.GenerativeModel('gemini-1.5-pro-001')

In [4]:
def convert_png_to_jpeg(png_path, jpeg_path):
    img = PIL.Image.open(png_path)
    rgb_img = img.convert('RGB')
    rgb_img.save(jpeg_path, 'JPEG')
    
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

import openai
import PIL
from PIL import Image

def language_models(system_text, user_text, image, use_openai=True, use_anthropic=True, use_google=True, max_tokens=1024):
    """
    Generates responses from multiple language models based on the given system text, user text, and image.

    Args:
        user_text (str): The user text representing the user's input.
        image (str): The path to the image file.
        use_openai (bool, optional): Whether to use the OpenAI model. Defaults to True.
        use_anthropic (bool, optional): Whether to use the Anthropic model. Defaults to True.
        use_google (bool, optional): Whether to use the Google model. Defaults to True.
        max_tokens (int, optional): The maximum number of tokens to generate in the response. Defaults to 1024.

    Returns:
        tuple: A tuple containing the responses from different language models.
            - chat_gpt_response (str): The response generated by the OpenAI GPT-4 Turbo model.
            - claude_response (str): The response generated by the Anthropic Claude model.
            - gemini_response (str): The response generated by the Google Gemini model.
    """
    
    chat_gpt_response, claude_response, gemini_response = None, None, None
    
    if os.path.isfile(image) == False:
        
        if use_openai:
            try:
                response = openai.chat.completions.create(
                    model="gpt-4o-2024-05-13",
                    max_tokens=max_tokens,
                    messages=[
                    {"role": "system", "content": system_text},
                    {"role": "user", "content": [
                        {
                        "type": "text",
                        "text": user_text
                        },
                        ]}],
                    
            )
                chat_gpt_response = response.choices[0].message.content
            except Exception as e:
                print(f"Error with OpenAI API: {e}")
            
        if use_anthropic:
            try:
                response = claude.messages.create(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=max_tokens,
                    system=system_text,
                    messages=[
                        {"role": "user", "content": [
                                {
                                    "type": "text",
                                    "text": user_text
                                }
                            ],
                        }
                    ],
                )
                claude_response = response.content[0].text
            except Exception as e:
                print(f"Error with Anthropic API: {e}")
    
        if use_google:
                try:
                    response = gemini.generate_content([system_text+ ' ' +user_text])
                    response.resolve()
                    gemini_response = response.text
                except Exception as e:
                    print(f"Error with Google API: {e}")
           
    else:
        try:
            convert_png_to_jpeg(image+'.png', image+'.jpeg')
        except Exception as e:
            print(f"Error converting image: {e}")
    
        try:
            base64_image = encode_image(image+'.jpeg')
            pil_image = PIL.Image.open(image+'.jpeg')
        except Exception as e:
            print(f"Error encoding image or opening image file: {e}")
        
        if use_openai:
            try:
                response = openai.chat.completions.create(
                    model="gpt-4o-2024-05-13",
                    max_tokens=max_tokens,
                    messages=[
                    {"role": "system", "content": system_text},
                    {"role": "user", "content": [
                        {
                        "type": "text",
                        "text": user_text
                        },
                        {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                        }]}],
                    
            )
                chat_gpt_response = response.choices[0].message.content
            except Exception as e:
                print(f"Error with OpenAI API: {e}")

        if use_anthropic:
            try:
                response = claude.messages.create(
                    model="claude-3-5-sonnet-20240620",
                    max_tokens=max_tokens,
                    system=system_text,
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                {
                                    "type": "image",
                                    "source": {
                                        "type": "base64",
                                        "media_type": "image/jpeg",
                                        "data": base64_image,
                                    },
                                },
                                {
                                    "type": "text",
                                    "text": user_text
                                }
                            ],
                        }
                    ],
                )
                claude_response = response.content[0].text
            except Exception as e:
                print(f"Error with Anthropic API: {e}")
    
        if use_google:
            try:
                response = gemini.generate_content([system_text+ ' ' +user_text, pil_image])
                response.resolve()
                gemini_response = response.text
            except Exception as e:
                print(f"Error with Google API: {e}")
    
    return chat_gpt_response, claude_response, gemini_response

In [5]:
system_text = 'Imagine that you are the best medical student. You want to excel in your medical examination! You will be presented with the following state questions. Only one answer is correct at a time. Only give back the correct answer letter as the solution, without an explanation!'

In [6]:
df = pd.read_excel('data/data_table_questions_new_modified.xlsx', sheet_name='Step 2')
df = df[['id', 'subject', 'question_type', 'question_number', 'total_question', 'correct_answer', ]]
df['id'] = df['id'].astype(int)
df['question_type'] = df['question_type'].astype(bool)

In [ ]:
# Initialize the results DataFrame
results = pd.DataFrame(columns=['id', 'correct_response', 'chat_gpt_response', 'claude_response', 'gemini_response'])

# Assuming df is already defined and contains the necessary columns
save_interval = 25
file_counter = 1

for i, id in enumerate(tqdm(range(837,1055), desc="Processing IDs")):  # range(1, 3) / df['id'] / [1, 17, 40, 136]
    
    user_text = df['total_question'][df['id'] == id].values[0]
    
    folder = df.iloc[id]['subject']
    file = df['question_number'][df['id'] == id].values[0]
    
    chat_gpt_response, claude_response, gemini_response = language_models(system_text, user_text, f'data/images_3/{folder}/{file}', max_tokens=1024, use_openai=False, use_anthropic=True, use_google=False)

    results = results._append({'id': id, 'correct_response': df['correct_answer'][df['id'] == id].values[0], 'chat_gpt_response': chat_gpt_response, 'claude_response': claude_response, 'gemini_response': gemini_response}, ignore_index=True)
    
    # Save the results to an Excel file every 50 iterations
    if (i + 1) % save_interval == 0:
        results.to_excel(f'results/results-usmle-{file_counter}.xlsx', index=False)
        file_counter += 1

# Save any remaining results after the loop
if not results.empty:
    results.to_excel(f'results/results-usmle-final.xlsx', index=False)


Processing IDs: 100%|██████████| 218/218 [04:02<00:00,  1.11s/it]
